## **Web - Django e Flask**

### **Flask** 

#### **1. Criação de uma aplicação Flask básica**

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **2. Renderização de templates usando Jinja2**

*Este código requer um arquivo `index.html` na pasta `templates`.*

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html', titulo='Página Inicial', conteudo='Olá, Mundo!')

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **3. Manipulação de formulários com Flask-WTF**

*Este código requer a instalação de `Flask-WTF` e um arquivo `formulario.html` na pasta `templates`.*

In [ ]:
from flask import Flask, render_template, request
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta'

class MeuFormulario(FlaskForm):
    nome = StringField('Nome')
    enviar = SubmitField('Enviar')

@app.route('/', methods=['GET', 'POST'])
def index():
    form = MeuFormulario()
    if form.validate_on_submit():
        nome = form.nome.data
        return f'Olá, {nome}!'
    return render_template('formulario.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **4. Autenticação de usuários com Flask-Login**

*Este código requer a instalação de `Flask-Login`.*

In [ ]:
from flask import Flask, render_template, redirect, url_for
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta'

login_manager = LoginManager()
login_manager.init_app(app)

class Usuario(UserMixin):
    def __init__(self, id):
        self.id = id

@login_manager.user_loader
def carregar_usuario(id):
    return Usuario(id)

@app.route('/login')
def login():
    user = Usuario(1)
    login_user(user)
    return redirect(url_for('index'))

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return 'Deslogado com sucesso!'

@app.route('/')
@login_required
def index():
    return 'Você está logado!'

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **5. Criação de APIs RESTful com Flask-RESTful**

*Este código requer a instalação de `Flask-RESTful`.*

In [ ]:
from flask import Flask
from flask_restful import Api, Resource

app = Flask(__name__)
api = Api(app)

class HelloWorld(Resource):
    def get(self):
        return {'message': 'Hello, World!'}

api.add_resource(HelloWorld, '/')

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **6. Armazenamento de sessões de usuário com Flask-Session**

*Este código requer a instalação de `Flask-Session`.*

In [ ]:
from flask import Flask, session, redirect, url_for

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta'
app.config['SESSION_TYPE'] = 'filesystem' # Armazena sessões em arquivos

@app.route('/login/<username>')
def login(username):
    session['username'] = username
    return redirect(url_for('index'))

@app.route('/')
def index():
    if 'username' in session:
        return f'Bem-vindo, {session["username"]}!'
    return 'Você não está logado!'

@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **7. Envio de e-mails com Flask-Mail**

*Requer a instalação de `Flask-Mail` e um arquivo `formulario_email.html`.*

In [ ]:
from flask import Flask, render_template, request
from flask_mail import Mail, Message

app = Flask(__name__)
# ATENÇÃO: Configure com suas credenciais de e-mail reais
app.config['SECRET_KEY'] = 'chave_secreta'
app.config['MAIL_SERVER'] = 'smtp.example.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USERNAME'] = 'seu_email@example.com'
app.config['MAIL_PASSWORD'] = 'sua_senha'
app.config['MAIL_USE_TLS'] = True

mail = Mail(app)

@app.route('/enviar-email', methods=['GET', 'POST'])
def enviar_email():
    if request.method == 'POST':
        destinatario = request.form['destinatario']
        assunto = request.form['assunto']
        corpo = request.form['corpo']
        msg = Message(assunto, sender='seu_email@example.com', recipients=[destinatario])
        msg.body = corpo
        mail.send(msg)
        return 'Email enviado com sucesso!'
    return render_template('formulario_email.html')

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **8. Autenticação OAuth com Flask-OAuthlib**

*Requer a instalação de `Flask-OAuthlib` e credenciais OAuth do provedor (ex: Google).*

In [ ]:
from flask import Flask, redirect, url_for, session
from flask_oauthlib.client import OAuth

app = Flask(__name__)
# ATENÇÃO: Configure com suas credenciais reais do Google
app.config['SECRET_KEY'] = 'chave_secreta'
app.config['OAUTH_CREDENTIALS'] = {
    'google': {
        'id': 'seu_id_de_cliente',
        'secret': 'seu_segredo_de_cliente'
    }
}

oauth = OAuth(app)
google = oauth.remote_app(
    'google',
    consumer_key=app.config['OAUTH_CREDENTIALS']['google']['id'],
    consumer_secret=app.config['OAUTH_CREDENTIALS']['google']['secret'],
    request_token_params={'scope': 'email'},
    base_url='https://www.googleapis.com/oauth2/v1/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://accounts.google.com/o/oauth2/token',
    authorize_url='https://accounts.google.com/o/oauth2/auth>',
)

@app.route('/')
def index():
    if 'google_token' in session:
        user_info = google.get('userinfo')
        return f'Usuário logado com Google: {user_info.data["email"]}'
    return redirect(url_for('login'))

@app.route('/login')
def login():
    return google.authorize(callback=url_for('authorized', _external=True))

@app.route('/logout')
def logout():
    session.pop('google_token', None)
    return redirect(url_for('index'))

@app.route('/login/authorized')
def authorized():
    resp = google.authorized_response()
    if resp is None or resp.get('access_token') is None:
        return 'Acesso negado'
    session['google_token'] = (resp['access_token'], '')
    return redirect(url_for('index'))

@google.tokengetter
def get_google_oauth_token():
    return session.get('google_token')

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **9. Cache de resposta com Flask-Caching**

*Requer a instalação de `Flask-Caching`.*

In [ ]:
from flask import Flask
from flask_caching import Cache
import time

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta'
app.config['CACHE_TYPE'] = 'SimpleCache' # Cache simples em memória

cache = Cache(app)

@app.route('/resultado')
@cache.cached(timeout=60) # Cache de 60 segundos
def resultado():
    # Simula uma operação demorada
    time.sleep(5)
    return f'Resultado gerado às {time.ctime()}'

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **10. Internacionalização (i18n) com Flask-Babel**

*Requer a instalação de `Flask-Babel`.*

In [ ]:
from flask import Flask, render_template, request
from flask_babel import Babel

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta'
app.config['BABEL_DEFAULT_LOCALE'] = 'pt'

babel = Babel(app)

@babel.localeselector
def get_locale():
    # Retorna o melhor idioma com base nas preferências do navegador
    return request.accept_languages.best_match(['en', 'es', 'pt'])

@app.route('/')
def index():
    # O template 'index.html' deve usar a função gettext (_), ex: <h1>{{ _('Olá Mundo') }}</h1>
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **11. Criação de Cookies e Sessões com Flask**

In [ ]:
from flask import Flask, request, session, make_response

app = Flask(__name__)
app.secret_key = 'uma_chave_muito_secreta'

@app.route('/session')
def exemplo_session():
    # Session: os dados são armazenados no servidor e um cookie seguro é enviado ao cliente.
    # Fecha junto com o navegador, a menos que seja configurada como permanente.
    session['usuario'] = 'Leonardo'
    return f'Sessão criada para o usuário: {session["usuario"]}'

@app.route('/cookie')
def exemplo_cookie():
    # Cookie: os dados são armazenados diretamente no navegador do cliente.
    resp = make_response('Cookie criado com sucesso!')
    resp.set_cookie('NomeUsuario', 'Leonardo')
    return resp

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **12. Manipulação de uploads de arquivos com Flask-Uploads**

*Requer a instalação de `Flask-Uploads` e a criação de uma pasta `uploads`.*

In [ ]:
from flask import Flask, request, render_template_string
from flask_uploads import UploadSet, configure_uploads, ALL

app = Flask(__name__)
# Configura o destino dos uploads para uma pasta chamada 'uploads'
app.config['UPLOADED_PHOTOS_DEST'] = 'uploads'
app.secret_key = 'chave_secreta_para_uploads'

photos = UploadSet('photos', ALL)
configure_uploads(app, photos)

@app.route('/upload', methods=['GET', 'POST'])
def upload_arquivo():
    if request.method == 'POST' and 'photo' in request.files:
        filename = photos.save(request.files['photo'])
        return f'Arquivo {filename} enviado com sucesso!'
    return render_template_string('''
        <!doctype html>
        <title>Upload de Arquivo</title>
        <h1>Faça o upload de um novo arquivo</h1>
        <form method=post enctype=multipart/form-data>
          <input type=file name=photo>
          <input type=submit value=Upload>
        </form>
    ''')

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **13. Criação de APIs RESTful com Flask-RESTPlus**

*Requer a instalação de `flask-restplus`.*

In [ ]:
from flask import Flask
from flask_restplus import Api, Resource

app = Flask(__name__)
api = Api(app, version='1.0', title='Minha API', description='Uma API de exemplo')

ns = api.namespace('exemplo', description='Operações de exemplo')

@ns.route('/hello')
class HelloWorld(Resource):
    def get(self):
        '''Retorna uma mensagem de olá mundo'''
        return {'message': 'Hello, World!'}

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **14. Manipulação de requisições com Flask-JWT**

*Requer a instalação de `Flask-JWT`.*

In [ ]:
from flask import Flask, jsonify, request
from flask_jwt import JWT, jwt_required, current_identity

class User:
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password

users = [
    User(1, 'usuario1', 'senha123'),
]

username_table = {u.username: u for u in users}
userid_table = {u.id: u for u in users}

def authenticate(username, password):
    user = username_table.get(username, None)
    if user and user.password == password:
        return user

def identity(payload):
    user_id = payload['identity']
    return userid_table.get(user_id, None)

app = Flask(__name__)
app.config['SECRET_KEY'] = 'chave_secreta_jwt'
jwt = JWT(app, authenticate, identity)

@app.route('/protegido')
@jwt_required()
def protegido():
    return f'Olá, {current_identity.username}! Você está acessando um recurso protegido.'

if __name__ == '__main__':
    app.run(debug=True)

-----

#### **15. Criação de páginas web estáticas com Flask-FlatPages**

*Requer a instalação de `Flask-FlatPages` e `Markdown`.*

In [ ]:
from flask import Flask, render_template
from flask_flatpages import FlatPages

# Crie uma pasta 'pages' e dentro dela um arquivo 'index.md'
# Exemplo de index.md:
# title: Minha Página Inicial
#
# # Bem-vindo!
# Este é um exemplo de página estática.

app = Flask(__name__)
app.config['FLATPAGES_EXTENSION'] = '.md'
app.config['FLATPAGES_ROOT'] = 'pages'
pages = FlatPages(app)

@app.route('/')
def index():
    page = pages.get('index')
    return render_template('page.html', page=page) # Requer um template 'page.html'

if __name__ == '__main__':
    app.run(debug=True)

-----

-----

### **Django** 🏗️

#### **16. Criação de um projeto Django básico**

*Estes são comandos de terminal e trechos de código para diferentes arquivos.*

**No terminal:**

```bash
django-admin startproject meu_projeto
cd meu_projeto
python manage.py startapp meu_app
```

**Em `meu_app/views.py`:**

In [ ]:
from django.http import HttpResponse

def index(request):
    return HttpResponse("Olá, mundo! Você está na página inicial.")

**Em `meu_app/urls.py` (crie este arquivo):**

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('', views.index, name='index'),
]

**Em `meu_projeto/urls.py`:**

In [ ]:
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('meu_app/', include('meu_app.urls')), # Inclui as URLs do app
]

-----

#### **17. Criação de modelos de banco de dados**

*Coloque este código em `meu_app/models.py`.*

In [ ]:
from django.db import models
from django.contrib.auth.models import User # Supondo que você use o usuário padrão do Django

class Pessoa(models.Model):
    nome = models.CharField(max_length=100)
    idade = models.IntegerField()
    email = models.EmailField()

    def __str__(self):
        return self.nome

class Post(models.Model):
    autor = models.ForeignKey(User, on_delete=models.CASCADE)
    titulo = models.CharField(max_length=200)
    conteudo = models.TextField()
    data_publicacao = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return self.titulo

-----

#### **18. Criação de formulários com Django Forms**

*Coloque este código em `meu_app/forms.py` (crie este arquivo).*

In [ ]:
from django import forms

class MeuFormulario(forms.Form):
    nome = forms.CharField(label='Seu Nome', max_length=100)
    email = forms.EmailField(label='Seu Email')
    mensagem = forms.CharField(label='Mensagem', widget=forms.Textarea)

-----

#### **19. Autenticação de usuários com Django Authentication System**

*Este é um exemplo de uso em uma view (`meu_app/views.py`).*

In [ ]:
from django.contrib.auth.models import User
from django.contrib.auth import authenticate, login, logout
from django.shortcuts import redirect

# Função de exemplo para registrar
def registrar_usuario(request, username, email, password):
    user = User.objects.create_user(username, email, password)
    login(request, user) # Opcional: logar após o registro
    return redirect('pagina_inicial') # Redireciona para alguma página

# Função de exemplo para login
def fazer_login(request, username, password):
    user = authenticate(request, username=username, password=password)
    if user is not None:
        login(request, user)
        return redirect('pagina_inicial')
    else:
        # Retorna uma mensagem de erro
        pass

# Função de exemplo para logout
def fazer_logout(request):
    logout(request)
    return redirect('pagina_inicial')

-----

#### **20. Criação de APIs RESTful com Django REST Framework**

*Requer a instalação de `djangorestframework`.*

**Em `meu_app/serializers.py` (crie este arquivo):**

In [ ]:
from rest_framework import serializers
from .models import Pessoa

class PessoaSerializer(serializers.ModelSerializer):
    class Meta:
        model = Pessoa
        fields = ['id', 'nome', 'idade', 'email']

**Em `meu_app/views.py`:**

In [ ]:
from rest_framework import viewsets
from .models import Pessoa
from .serializers import PessoaSerializer

class PessoaViewSet(viewsets.ModelViewSet):
    queryset = Pessoa.objects.all()
    serializer_class = PessoaSerializer

**Em `meu_projeto/urls.py`:**

In [ ]:
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from meu_app import views

router = DefaultRouter()
router.register(r'pessoas', views.PessoaViewSet)

urlpatterns = [
    # ... suas outras urls
    path('api/', include(router.urls)),
]

-----

#### **21. Criação de views usando Django Class-Based Views**

*Coloque em `meu_app/views.py`.*

In [ ]:
from django.views import View
from django.http import HttpResponse

class MinhaView(View):
    def get(self, request):
        return HttpResponse('Olá, mundo! Esta é uma view baseada em classes.')

-----

#### **22. Manipulação de formulários usando Django ModelForms**

*Coloque em `meu_app/forms.py`.*

In [ ]:
from django import forms
from .models import Pessoa

class PessoaForm(forms.ModelForm):
    class Meta:
        model = Pessoa
        fields = ['nome', 'idade', 'email']

-----

#### **23. Criação de middleware personalizado em Django**

*Crie um arquivo `meu_app/middleware.py`.*

In [ ]:
class MeuMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        # Código executado antes da view ser chamada
        print("Antes da view")

        response = self.get_response(request)

        # Código executado após a view ser chamada
        print("Depois da view")
        return response

*Não se esqueça de adicionar `'meu_app.middleware.MeuMiddleware'` à lista `MIDDLEWARE` em `settings.py`.*

-----

#### **24. Definir modelos em um aplicativo Django**

*Exemplo para `meu_app/models.py`.*

In [ ]:
from django.db import models

class MeuModelo(models.Model):
    campo1 = models.CharField(max_length=100)
    campo2 = models.IntegerField(default=0)
    campo3 = models.DateField(auto_now_add=True)

    def __str__(self):
        return self.campo1

-----

#### **25. Criar views em um aplicativo Django**

*Exemplo para `meu_app/views.py`.*

In [ ]:
from django.shortcuts import render
from .models import MeuModelo

def minha_view(request):
    objetos = MeuModelo.objects.all()
    context = {'objetos': objetos}
    return render(request, 'meu_app/minha_template.html', context)

-----

#### **26. Criar URLs em um aplicativo Django**

*Exemplo para `meu_app/urls.py`.*

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('meus-objetos/', views.minha_view, name='minha_view'),
]

-----

#### **27. Pré-Set para `CharField`**

*Exemplo de um campo de texto com tamanho máximo em `models.py`.*

In [ ]:
from django.db import models

class MeuModelo(models.Model):
    # Campo para armazenar strings de até 100 caracteres.
    meu_charfield = models.CharField(max_length=100)

-----

#### **28. Pré-Set para `ManyToManyField`**

*Exemplo de uma relação "muitos para muitos" em `models.py`.*

In [ ]:
from django.db import models

class Categoria(models.Model):
    nome = models.CharField(max_length=100)

class Produto(models.Model):
    nome = models.CharField(max_length=100)
    # Um produto pode ter várias categorias, e uma categoria pode ter vários produtos.
    categorias = models.ManyToManyField(Categoria)

-----

#### **29. Pré-Set para `OneToOneField`**

*Exemplo de uma relação "um para um" em `models.py`.*

In [ ]:
from django.db import models
from django.contrib.auth.models import User

class PerfilUsuario(models.Model):
    # Cada usuário tem apenas um perfil, e cada perfil pertence a apenas um usuário.
    usuario = models.OneToOneField(User, on_delete=models.CASCADE)
    bio = models.TextField(blank=True)
    foto_perfil = models.ImageField(upload_to='fotos_perfil', blank=True)

-----

#### **30. Pré-Set para `ForeignKey`**

*Exemplo de uma relação "muitos para um" em `models.py`.*

In [ ]:
from django.db import models

class Autor(models.Model):
    nome = models.CharField(max_length=100)

class Livro(models.Model):
    titulo = models.CharField(max_length=100)
    # Vários livros podem pertencer a um único autor.
    autor = models.ForeignKey(Autor, on_delete=models.CASCADE)